# ¡Hola mundo! del machine learning
Considero que el [hola mundo](https://es.wikipedia.org/wiki/Hola_mundo) del machine learning es la [regresión lineal](https://es.wikipedia.org/wiki/Regresi%C3%B3n_lineal).

El **modelo** resultante será la recta que mejor se **ajuste** a nuestro **dataset** o conjunto de datos, sklearn tiene [algunos](https://scikit-learn.org/stable/datasets/index.html), elegiremos el de **Boston house prices**, que tiene datos y valores (en miles) de varias viviendas en los suburbios... de Boston.

In [ ]:
# ¡venga, dataset!
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

Features en X, targets en y, así nombraremos las variables desde ahora.

La lista completa de las 13 features está [aquí](https://scikit-learn.org/stable/datasets/index.html#boston-house-prices-dataset) pero sólo usaremos la sexta **RM** promedio de habitaciones por vivienda.

Veamos cómo se relaciona con el precio gracias a una [gráfica de dispersion](https://es.wikipedia.org/wiki/Diagrama_de_dispersi%C3%B3n) realizada con [matplotlib](https://matplotlib.org/), una librería para graficar datos.

In [ ]:
import matplotlib.pyplot as plt

# seleccionamos todas las filas, sexta columna
X_RM = X[:, 5]
X_RM = X_RM.reshape(-1, 1)

plt.scatter(X_RM, y)
plt.xlabel("RM")
plt.ylabel("Valor")
plt.title("Relación entre RM y Valor")

En general, vemos una relación directamente proporcional: a más habitaciones, mayor costo.  
También vemos muchos puntos en la parte superior, esto parece indicar que los precios mayores a 50 mil han sido truncados a 50 mil.

In [ ]:
from sklearn.linear_model import LinearRegression

# ajustamos la recta
model = LinearRegression()
model.fit(X_RM, y)

print(f"Los parámetros obtenidos son: {model.coef_[0] :.2f} y {model.intercept_ :.2f}")

La pendiente es positiva, esto confirma la relación directamente proporcional.

# Utilizando el modelo
Con el modelo entrenado, podemos **estimar** o **predecir** el valor de todas las viviendas, sólo debemos llamar al método `predict`. Este usará internamente los parámetros `coef_` e `intercept` para la ecuación que ya conocemos.

Para evaluar qué tan bien predice el modelo comparemos la primera predicción con el valor real de la primera vivienda. Los valores reales son los targets, recuerda que están en y.

In [ ]:
predicted_values = model.predict(X_RM)

# comparemos con el valor real
print(f"El valor real es {y[0]}, y según el modelo es {predicted_values[0] :.2f}")

Hay una diferencia de -1.518 para esta vivienda, el modelo estima que el valor debe ser más alto porque la recta está encima del valor real, por supuesto, la diferencia será positiva de lo contrario y será mayor mientras más lejos esté de la recta.

Veamos cómo quedó la recta, tomaremos los valores máximo y mínimo de X, luego aplicamos la ecuación de la recta, tendremos dos puntos descritos por pares (X, y) que uniremos con un un gráfico de lineas usando la función `plot` de matplotlib:

In [ ]:
import numpy as np

plt.scatter(X_RM, y)

xpoints = [min(X_RM), max(X_RM)]
ypoints = model.predict(xpoints)

plt.plot(xpoints, ypoints, color="red")

Considerando que los valores son altos (hasta 50) no debemos preocuparnos mucho por un pequeño error de -1.518.  
Pero aplica sólo a la primera vivienda, ¿cómo evaluamos el error de todas?

# Evaluando el modelo
Podríamos promediar todos los errores, pero los que estén debajo de la resta serán **negativos** para solucionar esto podemos elevar cada error al cuadrado, sumar todos y dividir el resultado entre el número de errores.

$$\frac{1}{n} \sum{(y_{r} - \widehat{y} )^2}$$


$\widehat{y}$ se lee **y hat** (en español, *ye sombrero*) es el valor que predice el modelo, esto nos muestra que medimos la distancia vertical de cada punto a la recta para calcular el error. La recta que **mejor se ajusta** es aquella que **minimiza las distancias** hacia todos los puntos.

Esta fórmula es una **métrica** conocida como **mean squared error** (**MSE** para los amigos) o error cuadrático promedio.

In [ ]:
y_hat = predicted_values

squared_errors = (y_hat - y) ** 2
mse = sum(squared_errors) / len(y)

print(f"MSE: {mse :.2f}")

¡Es bastante alto!

Si un error es alto, este al cuadrado es mucho más alto, y podemos observar **outliers** o valores extremos que se alejan bastante de la recta, sumándolos... tenemos el porqué ese error tan alto.  
Es frecuente calcular el **square root of MSE (RMSE)** o raíz cuadrada del MSE para tener una mejor idea del error real, sin embargo este sigue siendo afectado por los outliers.

In [ ]:
rmse = mse ** (1/2)
print(f"RMSE: {rmse :.2f}")

Mucho mejor :)

Pero existe una métrica que no es afectada por outliers, **mean absolute error (MAE)** o error absoluto promedio, es otra solución para los errores negativos, quizá hayas pensado también en el valor absoluto.
Por cierto, tanto MSE como MAE se pueden calcular con `sklearn.metrics`

In [ ]:
from sklearn.metrics import mean_absolute_error

print(f"MAE: {mean_absolute_error(y_hat, y) :.2f}")

# Incluyendo más features
¿Recuerdas que sklearn espera un array 2D? cada feature que incluímos es una columna más, y tendremos los siguientes cambios:
- Entrenaremos una regresión lineal **múltiple**
- el atributo `.coef_` del modelo tendrá n features, siempre los tuvo, pero sólo tomamos el primer valor ;)
- es posible que mejore el modelo, si las features aportan a la predicción

Incluyamos la octava feature: **DIS** que describe la "distancia ponderada a cinco centros de empleo", ambas features estarán en una lista.

**TIP:** Si incluyes una sóla feature en la lista ya no necesitarás hacer el reshape a (-1, 1) ;D

In [ ]:
# todas las filas, sexta y octava columna
X_selected_features = X[:, [5, 7]]
X_selected_features

In [ ]:
# entrenamos otro modelo
model_selected_features = LinearRegression()

# ¡ya no es necesario reshape!
model_selected_features.fit(X_selected_features, y)

print("Coeficientes: ", model_selected_features.coef_)

# evaluemos
y_hat = model_selected_features.predict(X_selected_features)
print(f"MAE: {mean_absolute_error(y_hat, y) :.2f}")

¡Mejoró!

Intenta cambiar las features o incluir más, pero no todas las combinaciones darán un mejor modelo, por razones que trataremos más adelante.  
Tampoco podrás graficar en 2D si escoges 2 o más features, para ver cómo se ajustó la recta, pero siempre podras usar alguna métrica

# Regresión lineal múltiple
Como mencionamos, el modelo cambia de nombre ahora permite aprender de múltiples features.  
Un nuevo nombre conlleva una (casi) nueva ecuación:

$y = a_1x_1 + a_2x_2 + ... + a_nx_n + b $

+ n el número de features
+ a es un vector de coeficientes
+ x es el vector de cada feature
+ b es el término independiente

# ¿Es un buen modelo?
Las métricas son útiles, pero pueden ser engañosas, debemos escoger la que se adecue al problema, no la que menor error tenga. Lo importante es preguntarte ¿debería tomar en cuenta los outliers?  
La respuesta dependerá de los objetivos del modelo, y este dependerá de los datos.


Las métricas suelen ser lo último que extraemos de un proyecto de machine learning, antes debemos [probarlo](./3_test_set).